# QCQMC

This notebooks demonstrates the high-level code interface used to run the quantum part of https://arxiv.org/pdf/2106.16235.pdf.

The code is organized into five steps, where the output of each step is a dataclass named `[Name]Data`. Each step also has one or more `Params` class with a `build_` function to construct the output data.  Each of the steps are demonstrated below.

In [ ]:
import dataclasses

def print_fields(x):
    """Helper function to inspect returned objects' fields."""
    for field in dataclasses.fields(x):
        if hasattr(field.type, '__name__'):
            t = field.type.__name__
        else:
            t = field.type
        print(f'{field.name}: {t}')

## Hamiltonian

The first step is to pick a molecule and generate or load its relevant physical properties, namely the Hamiltonian. Here we specify a 2-electron Fermi-Hubbard Hamiltonian in the sto3g basis. Integral data is stored in the data/ file in this repository.

The resulting data consists of the one- and two-body integrals and some energy quantities.

In [ ]:
from recirq.qcqmc.hamiltonian import LoadFromFileHamiltonianParams, build_hamiltonian_from_file
hamiltonian_params = LoadFromFileHamiltonianParams(
    name='4q_pp',
    integral_key='fh_sto3g',
    n_orb=2,
    n_elec=2,
)
hamiltonian_params

In [ ]:
hamiltonian_data = build_hamiltonian_from_file(hamiltonian_params)
print_fields(hamiltonian_data)

## Trial Wavefunction

Next, we specify a trial wavefunction. Here: we request a perfect pairing trial (using the specialized Params class) and don't include any heuristic layers (to keep the example simple and the runtime short).

The output data includes parameterized circuits and their parameters.

In [ ]:
from recirq.qcqmc.trial_wf import PerfectPairingPlusTrialWavefunctionParams, build_pp_plus_trial_wavefunction
trial_wf_params = PerfectPairingPlusTrialWavefunctionParams(
    name='4q_pp',
    hamiltonian_params=hamiltonian_params,
    heuristic_layers=(),
)
trial_wf_params

In [ ]:
trial_wf_data = build_pp_plus_trial_wavefunction(
    trial_wf_params, 
    dependencies={
        hamiltonian_params: hamiltonian_data
    }
)
print('--'*20)
print_fields(trial_wf_data)

## Blueprint

Next, we configure the shadow tomography strategy for measuring the trial wavefunction. We specify how many cliffords and how to generate them, i.e. the qubit partition.

The returned data is a compiled circuit with parameterized clifford suffixes and Cirq resolvers for efficient execution on a device.

In [ ]:
qubits = trial_wf_params.qubits_linearly_connected

In [ ]:
from recirq.qcqmc.blueprint import BlueprintParamsTrialWf, build_blueprint

blueprint_params = BlueprintParamsTrialWf(
    name='4q_pp',
    trial_wf_params=trial_wf_params,
    n_cliffords=100,
    qubit_partition=tuple((q,) for q in qubits),
)
blueprint_params

In [ ]:
blueprint_data = build_blueprint(
    blueprint_params,
    dependencies={
        trial_wf_params: trial_wf_data
    })
print_fields(blueprint_data)

## Experiment

Now, we're ready to execute circuits and gather samples. The experiment step has two versions: simulated or on a real device. In either case, we configure how many samples to collect and any runtime-specific parameters.

The returned data includes the experimental samples.

In [ ]:
from recirq.qcqmc.experiment import SimulatedExperimentParams, build_experiment
expt_params = SimulatedExperimentParams(
    name='4q_pp',
    blueprint_params=blueprint_params,
    n_samples_per_clifford=1_000,
    noise_model_name='None',
)
expt_params

In [ ]:
expt_data = build_experiment(
    expt_params,
    dependencies={
        blueprint_params: blueprint_data
})
print_fields(expt_data)

## Overlap Analysis

Finally, we reconstruct the wavefunction from our experiment data.

The returned data here are reconstructed wavefunctions suitable for exporting to iPie for classical QCQMC walker updates.

In [ ]:
from recirq.qcqmc.analysis import OverlapAnalysisParams, build_analysis
analysis_params = OverlapAnalysisParams(
    name='4q_pp',
    experiment_params=expt_params,
    k_to_calculate=(1, 2),
)
analysis_params

In [ ]:
analysis_data = build_analysis(analysis_params, dependencies={
    expt_params: expt_data,
    blueprint_params: blueprint_data,
    trial_wf_params: trial_wf_data,
})
print_fields(analysis_data)